In [103]:
import pandas as pd
from openpyxl import load_workbook
from openpyxl.styles import Font, PatternFill, Alignment
from openpyxl.utils import get_column_letter
import datetime
import locale
locale.setlocale(locale.LC_TIME, 'ru_RU.UTF-8')
from io import BytesIO

In [104]:
# Словарь месяцев вручную, чтобы избежать проблем с кодировкой
months_ru = {
    1: "янв", 2: "фев", 3: "мар", 4: "апр", 5: "май", 6: "июн",
    7: "июл", 8: "авг", 9: "сен", 10: "окт", 11: "ноя", 12: "дек"
}

In [108]:
df = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\112025_Продажи за октябрь.xlsx", sheet_name = 'рейтинг')

In [106]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 236 entries, 0 to 235
Data columns (total 2 columns):
 #   Column      Non-Null Count  Dtype  
---  ------      --------------  -----  
 0   Застройщик  236 non-null    object 
 1   Indicators  164 non-null    float64
dtypes: float64(1), object(1)
memory usage: 3.8+ KB


In [22]:
df

,Застройщик,Indicators
0,Самолет,1958.0
1,ПИК,1770.0
2,А101,683.0
3,ДСК-1 (ФСК Лидер),582.0
4,Гранель,546.0
...,...,...
231,Бэсткон,NaN
232,Инвестстройрегион,NaN
233,Меджиком,NaN
234,ПСТ,NaN


In [107]:
developers = df["Застройщик"].tolist()

In [79]:
df2 = pd.read_excel(r"C:\Users\m.olshanskiy\Desktop\112025_Продажи за октябрь.xlsx", sheet_name = 'массив')

In [109]:
def format_col(col):
    if isinstance(col, (pd.Timestamp, datetime.datetime)):
        return f"{months_ru[col.month]}.{str(col.year)[-2:]}"
    return col

df2.columns = [format_col(col) for col in df2.columns]


In [110]:
# Фильтруем по каждому региону
def remove_total_if_one_jk(group):
    # Проверяем сколько строк с ЖК (не Total)
    jk_count = group[group["Название ЖК"] != "Total"].shape[0]
    if jk_count <= 1:
        # Удаляем строки с Total
        group = group[group["Название ЖК"] != "Total"]
    return group

In [111]:
# Проверяем результат
print(df2.columns.tolist())

['№', 'Застройщик', 'Регион', 'Название ЖК', 'Среднее за 2024 г., шт', 'янв.25', 'фев.25', 'мар.25', 'апр.25', 'май.25', 'июн.25', 'июл.25', 'авг.25', 'сен.25', 'окт.25', 'Среднее за 2025 г., шт']


In [112]:
region_order = ['Total', 'Москва', 'Новая Москва', 'Московская область']

In [113]:
# 2. Создаём словарь: застройщик → порядковый номер
developer_numbers = {name: i + 1 for i, name in enumerate(df["Застройщик"].dropna().unique())}

# 3. Добавляем новый столбец с номерами
df2["№"] = df2["Застройщик"].map(developer_numbers)

# 4. Ставим столбец "№" в начало
df2 = df2[["№"] + [col for col in df2.columns if col != "№"]]





In [114]:
df2['Застройщик'] = pd.Categorical(df2['Застройщик'], categories=developers, ordered=True)
df2['Регион'] = pd.Categorical(df2['Регион'], categories=region_order, ordered=True)
df_sorted = df2.sort_values(['Застройщик', 'Регион']).reset_index(drop=True)

# 5. Дублируем шапку перед каждым застройщиком
header = pd.DataFrame([df_sorted.columns], columns=df_sorted.columns)  # создаём строку-шапку
result = pd.DataFrame(columns=df_sorted.columns)

for dev in df_sorted["№"].unique():
    block = df_sorted[df_sorted["№"] == dev]
    result = pd.concat([result, header, block], ignore_index=True)

df_sorted = result

# 5. Заменяем все значения "Total" на "Итого"
# df_sorted = df_sorted.replace("Total", "Итого")

df_cleaned = df_sorted.groupby("Регион", group_keys=False).apply(remove_total_if_one_jk)

C:\Users\m.olshanskiy\AppData\Local\Temp\ipykernel_2384\2155248353.py:18: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  df_cleaned = df_sorted.groupby("Регион", group_keys=False).apply(remove_total_if_one_jk)


In [116]:
df_sorted = df_cleaned

In [122]:
# --- 2. Сохраняем в Excel ---
output_file = r"C:\Users\m.olshanskiy\Desktop\Продажи отсортированные 12.11-2.xlsx"
df_sorted.to_excel(output_file, index=False)

# --- 3. Открываем для форматирования ---
wb = load_workbook(output_file)
ws = wb.active

# Эта часть удаляет строки Total там, где в регионе всего один проект

region_start = None
region_end = None
current_region = None
rows_to_delete = []

for row in range(2, ws.max_row + 1):  # первая строка — заголовок
    region = ws[f"C{row}"].value  # теперь регион в колонке C
    jk_name = ws[f"D{row}"].value  # Total / ЖК в колонке D

    if region is None:
        continue
    if jk_name is None:
        jk_name = ""

    if region != current_region:
        if region_start is not None:
            # Собираем все названия ЖК в регионе
            jk_list = [str(ws[f"D{r}"].value).strip() for r in range(region_start, region_end + 1)]
            total_count = sum(1 for name in jk_list if name.lower() == "total")
            real_jk_count = sum(1 for name in jk_list if name.lower() != "total")

            print(f"\nРегион: {current_region}")
            print(f"Список ЖК (включая Total): {jk_list}")
            print(f"Количество ЖК без Total: {real_jk_count}, Total: {total_count}")

            # Если ЖК всего 1, удаляем Total
            if real_jk_count == 1 and total_count > 0:
                for r in range(region_start, region_end + 1):
                    if str(ws[f"D{r}"].value).strip().lower() == "total":
                        rows_to_delete.append(r)
                        print(f"Удаляю строку {r} с 'Total'")

        current_region = region
        region_start = row

    region_end = row

# Проверяем последний регион
if region_start is not None:
    jk_list = [str(ws[f"D{r}"].value).strip() for r in range(region_start, region_end + 1)]
    total_count = sum(1 for name in jk_list if name.lower() == "total")
    real_jk_count = sum(1 for name in jk_list if name.lower() != "total")

    print(f"\nРегион: {current_region}")
    print(f"Список ЖК (включая Total): {jk_list}")
    print(f"Количество ЖК без Total: {real_jk_count}, Total: {total_count}")

    if real_jk_count == 1 and total_count > 0:
        for r in range(region_start, region_end + 1):
            if str(ws[f"D{r}"].value).strip().lower() == "total":
                rows_to_delete.append(r)
                print(f"Удаляю строку {r} с 'Total'")

# Удаляем строки с конца
for r in sorted(rows_to_delete, reverse=True):
    ws.delete_rows(r)

# Эта часть добавляет шапку для каждого застройщика, а также добавляет серую заливку и жирный шрифт

# Форматы
fill = PatternFill(start_color="D9D9D9", end_color="D9D9D9", fill_type="solid")
bold_font = Font(bold=True)
center_align = Alignment(vertical="center", horizontal="center")

# --- 4. Форматируем строки-шапки ---
for row in ws.iter_rows(min_row=1, max_row=ws.max_row):
    if row[0].value == "№":  # шапка начинается со "№"
        for cell in row:
            cell.font = bold_font
            cell.fill = fill
            cell.alignment = center_align

# --- 5. Объединяем одинаковые подряд ячейки ---
def merge_identical_cells(column_idx):
    start = 2  # пропускаем первую строку
    current_value = ws[f"{get_column_letter(column_idx)}{start}"].value
    for row in range(3, ws.max_row + 2):
        cell_value = ws[f"{get_column_letter(column_idx)}{row}"].value
        if cell_value != current_value:
            if row - start > 1 and current_value is not None:
                ws.merge_cells(start_row=start, start_column=column_idx,
                               end_row=row - 1, end_column=column_idx)
                ws[f"{get_column_letter(column_idx)}{start}"].alignment = center_align
            start = row
            current_value = cell_value

# Объединяем по нужным столбцам
merge_identical_cells(1)  # №
merge_identical_cells(2)  # Застройщик
merge_identical_cells(3)  # Регион
merge_identical_cells(4)  # Название ЖК (если нужно)




# --- 6. Сохраняем итог ---
wb.save(output_file)
print("✅ Готово! Шапки добавлены, выделены цветом, и ячейки объединены.")



Регион: Регион
Список ЖК (включая Total): ['Название ЖК']
Количество ЖК без Total: 1, Total: 0

Регион: Total
Список ЖК (включая Total): ['None']
Количество ЖК без Total: 1, Total: 0

Регион: Москва
Список ЖК (включая Total): ['Total', 'Молжаниново', 'Квартал Домашний', 'Верейская 41', 'Октябрьская 98', 'Квартал на воде', 'Нова', 'Стремянный 2']
Количество ЖК без Total: 7, Total: 1

Регион: Новая Москва
Список ЖК (включая Total): ['Total', 'Остафьево', 'Эко Бунино', 'Новое Внуково', 'Цветочные поляны Экопарк', 'Квартал Западный', 'Квартал Марьино', 'Тропарево Парк', 'Подольские кварталы', 'Алхимово', 'Цветочные поляны Сити', 'Квартал Румянцево', 'Ольховый квартал']
Количество ЖК без Total: 12, Total: 1

Регион: Московская область
Список ЖК (включая Total): ['Total', 'Пригород Лесное', 'Мытищи Парк', 'Новоград Павлино', 'Горки Парк', 'Новое Видное', 'Прибрежный парк', 'Рублевский квартал', 'Пятницкие луга', 'Квартал Ивакино', 'Томилино Парк', 'Егорово Парк', 'Богдановский Лес', 'Кварта